In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [16]:
# Загрузить данные из файлов
# matches706_team_picks = pd.read_csv('table/table with carry or support in 7.06.csv').drop(['Unnamed: 0'], axis=1)
# matches707_team_picks = pd.read_csv('table/table with carry or support in 7.07 to 29.12.17.csv', index_col=0)
matches_team_picks = pd.read_csv('table/table with carry or support  from 1.09.13 to 22.01.18.csv', index_col=0)
matches_team_picks.fillna(0)
# # выбросит команды
# matches706_team_picks = matches706_team_picks.drop(['radiant_id', 'dire_id'], axis=1)
# matches707_team_picks = matches707_team_picks.drop(['radiant_id', 'dire_id'], axis=1)
# # Соединить даныне
# matches_team_picks = matches706_team_picks.append(matches707_team_picks).reset_index().drop(['index'], axis=1)

#Разделение данных без перемешивания на traning & test для каждого патча и для соединенных данных
X_706, y_706 = np.split(matches706_team_picks, [len(matches706_team_picks) - int(len(matches706_team_picks)*0.3)]) 
X_707, y_707 = np.split(matches707_team_picks, [len(matches707_team_picks) - int(len(matches707_team_picks)*0.3)]) 
X, y = np.split(matches_team_picks, [len(matches_team_picks) - 400]) 

X_train = X.drop(['radiant_win'], axis=1)
y_train = X['radiant_win']

X_test = y.drop(['radiant_win'], axis=1)
y_test = y['radiant_win']

# X_train_706 = X_706.drop(['radiant_win'], axis=1)
# y_train_706 = X_706['radiant_win']

# X_test_706 = y_706.drop(['radiant_win'], axis=1)
# y_test_706 = y_706['radiant_win']

# X_train_707 = X_707.drop(['radiant_win'], axis=1)
# y_train_707 = X_707['radiant_win']

# X_test_707 = y_707.drop(['radiant_win'], axis=1)
# y_test_707 = y_707['radiant_win']


In [27]:

X_train = X_train.fillna(0)
index = X_train['Escape10'].index[X_train['Escape10'].apply(np.isnan)]
index

Int64Index([], dtype='int64')

In [32]:
# дерево 

tree = DecisionTreeClassifier(max_depth=13)

tree.fit(X_train, y_train)
print (tree.score(X_train, y_train))
print (tree.score(X_test, y_test))
print ('----------------------------')
# tree.fit(X_train_706, y_train_706)
# print (tree.score(X_train_706, y_train_706))
# print (tree.score(X_test_706, y_test_706))
# print ('----------------------------')
# tree.fit(X_train_707, y_train_707)
# print (tree.score(X_train_707, y_train_707))
# print (tree.score(X_test_707, y_test_707))

0.848119777159
0.5425
----------------------------


In [36]:
# создание большого кол-ва лесов с разными параметрами и запись Score в массив
from sklearn.ensemble import RandomForestClassifier
all_parametrs=[]
for n in range(50, 100, 5):
    for depth in range(15, 100, 5):
        for features in range(1, 103, 5):
            parametrs=[]
            forest_optim_param = RandomForestClassifier(n_estimators=n, random_state=42, 
                                                        max_depth=depth, max_features=features)
            forest_optim_param.fit(X_train_707, y_train_707)
            if (forest_optim_param.score(X_test_707, y_test_707) > 0.62):
                parametrs.append(n)
                parametrs.append(depth)
                parametrs.append(features)
                parametrs.append(forest_optim_param.score(X_train_707, y_train_707))
                parametrs.append(forest_optim_param.score(X_test_707, y_test_707))
                all_parametrs.append(parametrs)
    print (n)

In [10]:
all_test=[]

for par in all_parametrs:
    if par[4] > 0.6299:
        all_test.append(par)

In [14]:
all_test=[]
all_parametrs

[[50, 15, 51, 1.0, 0.62393162393162394],
 [50, 15, 81, 1.0, 0.62393162393162394],
 [50, 20, 41, 1.0, 0.62393162393162394],
 [50, 25, 41, 1.0, 0.62393162393162394],
 [50, 30, 41, 1.0, 0.62393162393162394]]

In [34]:
# Случайный лес с параметрами которые оказались лучшими на обучении патча7,06
forest_last = RandomForestClassifier(n_estimators=100, random_state=42, 
                                                        max_depth=25, max_features=60)
forest_last.fit(X_train, y_train)
print (forest_last.score(X_train, y_train))
print (forest_last.score(X_test, y_test))

# forest_last.fit(X_train_706, y_train_706)
# print (forest_last.score(X_train_706, y_train_706))
# print (forest_last.score(X_test_706, y_test_706))

# forest_last.fit(X_train_707, y_train_707)
# print (forest_last.score(X_train_707, y_train_707))
# print (forest_last.score(X_test_707, y_test_707))

1.0
0.5325


In [35]:
# Нейронные сети
mlp = MLPClassifier(solver='lbfgs', random_state=1, hidden_layer_sizes=[50, 50, 5])

mlp.fit(X_train, y_train)
print (mlp.score(X_train, y_train))
print (mlp.score(X_test, y_test))
# print ('----------------------------')
# mlp.fit(X_train_706, y_train_706)
# print (mlp.score(X_train_706, y_train_706))
# print (mlp.score(X_test_706, y_test_706))
# print ('----------------------------')
# mlp.fit(X_train_707, y_train_707)
# print (mlp.score(X_train_707, y_train_707))
# print (mlp.score(X_test_707, y_test_707))

0.864554317549
0.5175


In [37]:
# Градиентный бустинг
gbr = GradientBoostingClassifier(n_estimators=100, max_depth=15, max_features=80)
gbr.fit(X_train, y_train)
print (gbr.score(X_train, y_train))
print (gbr.score(X_test, y_test))
# print ('----------------------------')
# gbr.fit(X_train_706, y_train_706)
# print (gbr.score(X_train_706, y_train_706))
# print (gbr.score(X_test_706, y_test_706))
# print ('----------------------------')
# gbr.fit(X_train_707, y_train_707)
# print (gbr.score(X_train_707, y_train_707))
# print (gbr.score(X_test_707, y_test_707))

1.0
0.505
